# Getting Started

## construct `tensors`
Tensors are similar to NumPy’s ndarrays, with the addition being that Tensors can also be used on a GPU to accelerate computing.

In [1]:
import torch
x = torch.empty(3, 2)
print(x)

tensor([[ 0.0000e+00, -1.0842e-19],
        [-2.3897e+04,  4.6577e-10],
        [ 4.1345e+26,  4.5800e-41]])


In [2]:
torch.manual_seed(1234)
x = torch.rand(3, 2) # a uniform distribution on the interval [0, 1)
print(x)

tensor([[0.0290, 0.4019],
        [0.2598, 0.3666],
        [0.0583, 0.7006]])


In [3]:
x = torch.zeros(3, 2, dtype=torch.long)
print(x)

tensor([[0, 0],
        [0, 0],
        [0, 0]])


Construct a tensor directly from data:

In [4]:
x = torch.tensor([5.5, 3])
print(x)

tensor([5.5000, 3.0000])


or create a tensor based on an existing tensor. These methods will reuse properties of the input tensor, e.g. dtype, unless new values are provided by user

In [5]:
y = x.new_ones(3, 2, dtype=torch.double)      # new_* methods take in sizes
z = torch.randn_like(y, dtype=torch.float)    # override dtype!
print(y)
print(z) 

tensor([[1., 1.],
        [1., 1.],
        [1., 1.]], dtype=torch.float64)
tensor([[-0.8545,  0.5098],
        [-0.0821,  0.6607],
        [ 0.0785,  0.7884]])


In [6]:
# torch.Size is in fact a tuple, so it supports all tuple operations.
print(y.size(), z.size()) 

torch.Size([3, 2]) torch.Size([3, 2])


## Operations
[Full Reference](https://pytorch.org/docs/stable/torch.html)

In [20]:
x = torch.ones(4, 2, dtype=torch.long)
y = torch.ones(4, 2, dtype=torch.long) * 2
z = torch.add(x, y)
print(z)

tensor([[3, 3],
        [3, 3],
        [3, 3],
        [3, 3]])


Any operation that mutates a tensor in-place is post-fixed with an `_`. For example: `x.t_()`, will change `x`.

In [21]:
x = torch.ones(4, 2, dtype=torch.long)
x.size()

torch.Size([4, 2])

In [22]:
x.t_()
x.size()

torch.Size([2, 4])

You can use standard NumPy-like indexing with all bells and whistles!

In [25]:
z[:, [1]].size()

torch.Size([4, 1])

You can use `view` to reshape tensors.

In [27]:
x = torch.randn(4, 4)
y = x.view(16)
z = x.view(-1, 8)  # the size -1 is inferred from other dimensions
print(x.size(), y.size(), z.size())

torch.Size([4, 4]) torch.Size([16]) torch.Size([2, 8])


## Communicate with `Numpy`

### `tensor` `->` `array`

In [41]:
x_torch = torch.ones(5)
x_np = x_torch.numpy()

In [42]:
x_torch, x_np

(tensor([1., 1., 1., 1., 1.]), array([1., 1., 1., 1., 1.], dtype=float32))

In [43]:
# this will also change the value of x_np
x_torch.add_(2), x_np

(tensor([3., 3., 3., 3., 3.]), array([3., 3., 3., 3., 3.], dtype=float32))

### `array` `->` `tensor` 

In [45]:
import numpy as np
a = np.ones(5)
b = torch.from_numpy(a)
np.add(a, 1, out=a)
a,b

(array([2., 2., 2., 2., 2.]),
 tensor([2., 2., 2., 2., 2.], dtype=torch.float64))